In [1]:
import torch 
from torch import nn
import torch.nn.functional as F


import numpy as np

In [2]:
with open('happy_birthday.txt', 'r', encoding='utf8') as f:
    text = f.read()

In [3]:
print(len(text))

14854


In [4]:
all_characters = set(text)

# num -- > letter
decoder = dict(enumerate(all_characters))
# letter --> num
encoder = {char: index for index, char in decoder.items()}

# build list of all encoded chars for text
encoded_text = np.array([encoder[char] for char in text])

In [5]:
def one_hot_encoder(encoded_text, num_uni_chars):
    
    # encoded_text --> batch of encoded text
    # nm_uni_chars --> len(set(text))
    
    one_hot = np.zeros((encoded_text.size, num_uni_chars))
    one_hot = one_hot.astype(np.float32)
    one_hot[np.arange(one_hot.shape[0]), encoded_text.flatten()] = 1.0
    one_hot = one_hot.reshape((*encoded_text.shape, num_uni_chars))
    
    return one_hot

In [6]:
def generate_batches(encoded_text, samp_per_batch=10, seq_len=50):
    
    # X : encoded text of length seq_len
    # [0, 1, 2]
    # [1, 2, 3]
    
    # Y : encoded text shifted by one
    
    # [1, 2, 3]
    # [2, 3, 4]
    
    # how many chars per batch?
    char_per_batch = samp_per_batch * seq_len 
    # how many batches can we make, given the len of encoded text?
    num_batches_available = int(len(encoded_text) / char_per_batch)
    # cut off the end of the encoded text, that won't fit evenly into a batch
    encoded_text = encoded_text[ : num_batches_available * char_per_batch]
    # reshape the encoded text
    encoded_text = encoded_text.reshape((samp_per_batch, -1))
    
    for n in range(0, encoded_text.shape[1], seq_len):
        
        x = encoded_text[ : , n : n + seq_len]
        # zeros array to the same shape as x
        y = np.zeros_like(x)
        
        try:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, n+seq_len]
            
        except:
            y[:, :-1] = x[:, 1:]
            y[:, -1] = encoded_text[:, 0]
            
        yield x, y

In [7]:
class CharModel(nn.Module):
    
    def __init__(self, all_chars, num_hidden=256, num_layers=4,drop_prob=0.5,use_gpu=False):
        
        
        # SET UP ATTRIBUTES
        super().__init__()
        self.drop_prob = drop_prob
        self.num_layers = num_layers
        self.num_hidden = num_hidden
        self.use_gpu = use_gpu
        
        #CHARACTER SET, ENCODER, and DECODER
        self.all_chars = all_chars
        self.decoder = dict(enumerate(all_chars))
        self.encoder = {char: ind for ind,char in decoder.items()}
        
        
        self.lstm = nn.LSTM(len(self.all_chars), num_hidden, num_layers, dropout=drop_prob, batch_first=True)
        
        self.dropout = nn.Dropout(drop_prob)
        
        self.fc_linear = nn.Linear(num_hidden, len(self.all_chars))
      
    
    def forward(self, x, hidden):
                  
        lstm_output, hidden = self.lstm(x, hidden)
        
        drop_output = self.dropout(lstm_output)
        drop_output = drop_output.contiguous().view(-1, self.num_hidden)
        
        final_out = self.fc_linear(drop_output)

        return final_out, hidden
    
    
    def hidden_state(self, batch_size):
        '''
        Used as separate method to account for both GPU and CPU users.
        '''
        
        if self.use_gpu:
            
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda(),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden).cuda())
        else:
            hidden = (torch.zeros(self.num_layers,batch_size,self.num_hidden),
                     torch.zeros(self.num_layers,batch_size,self.num_hidden))
        
        return hidden
        

In [8]:
model = CharModel(all_chars=all_characters,
                 num_hidden=512,
                 num_layers=3,
                 drop_prob=0.5,
                 use_gpu=True)

In [9]:
total_param = []

for p in model.parameters():
    total_param.append(int(p.numel()))
    
total_param

[155648,
 1048576,
 2048,
 2048,
 1048576,
 1048576,
 2048,
 2048,
 1048576,
 1048576,
 2048,
 2048,
 38912,
 76]

In [10]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss()

In [11]:
train_percent = 0.9
train_index = int(len(encoded_text) * train_percent)

train_data = encoded_text[:train_index]
val_data = encoded_text[train_index:]

In [12]:
# Variables

epochs = 100
batch_size = 10
seq_len = 10
tracker = 0
num_char = max(encoded_text) + 1

previous_loss = 100
min_loss = 100

In [13]:
# Set model to train
model.train()


# Check to see if using GPU
if model.use_gpu:
    model.cuda()

for i in range(epochs):
    
    hidden = model.hidden_state(batch_size)
    
    
    for x,y in generate_batches(train_data,batch_size,seq_len):
        
        tracker += 1
        
        # One Hot Encode incoming data
        x = one_hot_encoder(x,num_char)
        
        # Convert Numpy Arrays to Tensor
        
        inputs = torch.from_numpy(x)
        targets = torch.from_numpy(y)
        
        # Adjust for GPU if necessary
        
        if model.use_gpu:
            
            inputs = inputs.cuda()
            targets = targets.cuda()
            
        # Reset Hidden State
        # If we dont' reset we would backpropagate through all training history
        hidden = tuple([state.data for state in hidden])
        
        model.zero_grad()
        
        lstm_output, hidden = model.forward(inputs,hidden)
        loss = criterion(lstm_output,targets.view(batch_size*seq_len).long())
        
        loss.backward()
        
        # POSSIBLE EXPLODING GRADIENT PROBLEM!
        # LET"S CLIP JUST IN CASE
        nn.utils.clip_grad_norm_(model.parameters(),max_norm=5)
        
        optimizer.step()
        
        
        
        ###################################
        ### CHECK ON VALIDATION SET ######
        #################################
        
        if tracker % 25 == 0:
            
            val_hidden = model.hidden_state(batch_size)
            val_losses = []
            model.eval()
            
            for x,y in generate_batches(val_data,batch_size,seq_len):
                
                # One Hot Encode incoming data
                x = one_hot_encoder(x, num_char)
                

                # Convert Numpy Arrays to Tensor

                inputs = torch.from_numpy(x)
                targets = torch.from_numpy(y)

                # Adjust for GPU if necessary

                if model.use_gpu:

                    inputs = inputs.cuda()
                    targets = targets.cuda()
                    
                # Reset Hidden State
                # If we dont' reset we would backpropagate through 
                # all training history
                val_hidden = tuple([state.data for state in val_hidden])
                
                lstm_output, val_hidden = model.forward(inputs,val_hidden)
                val_loss = criterion(lstm_output,targets.view(batch_size*seq_len).long())
                val_losses.append(val_loss.item())
            
            # Reset to training model after val for loop
            model.train()
    
            current_loss = val_loss.item()
        
            print(f"Epoch: {i} Step: {tracker} \nCurrent Validation Loss: {current_loss} \nPrevious Loss: {previous_loss}")

            if (current_loss < previous_loss):
                if (current_loss < min_loss):
                    print('Error decreased!')
                    torch.save(model.state_dict(), 'min_hidden512_layers3_happy_birthday.net')
                    print('Saved model.')
                    min_loss = current_loss
                else:
                    print('Error increased!')
                
            previous_loss = val_loss.item()
            print()

Epoch: 0 Step: 25 
Current Validation Loss: 3.4874885082244873 
Previous Loss: 100
Error decreased!
Saved model.

Epoch: 0 Step: 50 
Current Validation Loss: 3.507577419281006 
Previous Loss: 3.4874885082244873

Epoch: 0 Step: 75 
Current Validation Loss: 3.472723960876465 
Previous Loss: 3.507577419281006
Error decreased!
Saved model.

Epoch: 0 Step: 100 
Current Validation Loss: 3.497837781906128 
Previous Loss: 3.472723960876465

Epoch: 0 Step: 125 
Current Validation Loss: 3.464243173599243 
Previous Loss: 3.497837781906128
Error decreased!
Saved model.

Epoch: 1 Step: 150 
Current Validation Loss: 3.396707773208618 
Previous Loss: 3.464243173599243
Error decreased!
Saved model.

Epoch: 1 Step: 175 
Current Validation Loss: 3.3431310653686523 
Previous Loss: 3.396707773208618
Error decreased!
Saved model.

Epoch: 1 Step: 200 
Current Validation Loss: 3.2351949214935303 
Previous Loss: 3.3431310653686523
Error decreased!
Saved model.

Epoch: 1 Step: 225 
Current Validation Loss: 3.1

Epoch: 13 Step: 1850 
Current Validation Loss: 2.798659563064575 
Previous Loss: 2.8296377658843994
Error increased!

Epoch: 14 Step: 1875 
Current Validation Loss: 2.784360647201538 
Previous Loss: 2.798659563064575
Error increased!

Epoch: 14 Step: 1900 
Current Validation Loss: 2.8119046688079834 
Previous Loss: 2.784360647201538

Epoch: 14 Step: 1925 
Current Validation Loss: 2.78642201423645 
Previous Loss: 2.8119046688079834
Error increased!

Epoch: 14 Step: 1950 
Current Validation Loss: 2.835395097732544 
Previous Loss: 2.78642201423645

Epoch: 14 Step: 1975 
Current Validation Loss: 2.800344228744507 
Previous Loss: 2.835395097732544
Error increased!

Epoch: 15 Step: 2000 
Current Validation Loss: 2.7806169986724854 
Previous Loss: 2.800344228744507
Error increased!

Epoch: 15 Step: 2025 
Current Validation Loss: 2.747572660446167 
Previous Loss: 2.7806169986724854
Error decreased!
Saved model.

Epoch: 15 Step: 2050 
Current Validation Loss: 2.7831521034240723 
Previous Loss: 

Epoch: 28 Step: 3725 
Current Validation Loss: 3.2739243507385254 
Previous Loss: 3.268458843231201

Epoch: 28 Step: 3750 
Current Validation Loss: 3.3639276027679443 
Previous Loss: 3.2739243507385254

Epoch: 28 Step: 3775 
Current Validation Loss: 3.3230860233306885 
Previous Loss: 3.3639276027679443
Error increased!

Epoch: 28 Step: 3800 
Current Validation Loss: 3.4323031902313232 
Previous Loss: 3.3230860233306885

Epoch: 28 Step: 3825 
Current Validation Loss: 3.4339139461517334 
Previous Loss: 3.4323031902313232

Epoch: 28 Step: 3850 
Current Validation Loss: 3.3979804515838623 
Previous Loss: 3.4339139461517334
Error increased!

Epoch: 29 Step: 3875 
Current Validation Loss: 3.219109296798706 
Previous Loss: 3.3979804515838623
Error increased!

Epoch: 29 Step: 3900 
Current Validation Loss: 3.4584927558898926 
Previous Loss: 3.219109296798706

Epoch: 29 Step: 3925 
Current Validation Loss: 3.468660831451416 
Previous Loss: 3.4584927558898926

Epoch: 29 Step: 3950 
Current Valid

Epoch: 42 Step: 5600 
Current Validation Loss: 3.897030830383301 
Previous Loss: 3.9979987144470215
Error increased!

Epoch: 42 Step: 5625 
Current Validation Loss: 4.019464492797852 
Previous Loss: 3.897030830383301

Epoch: 42 Step: 5650 
Current Validation Loss: 4.099851608276367 
Previous Loss: 4.019464492797852

Epoch: 42 Step: 5675 
Current Validation Loss: 3.993227005004883 
Previous Loss: 4.099851608276367
Error increased!

Epoch: 42 Step: 5700 
Current Validation Loss: 4.14841365814209 
Previous Loss: 3.993227005004883

Epoch: 43 Step: 5725 
Current Validation Loss: 4.217618465423584 
Previous Loss: 4.14841365814209

Epoch: 43 Step: 5750 
Current Validation Loss: 4.028300762176514 
Previous Loss: 4.217618465423584
Error increased!

Epoch: 43 Step: 5775 
Current Validation Loss: 4.058918476104736 
Previous Loss: 4.028300762176514

Epoch: 43 Step: 5800 
Current Validation Loss: 4.0021586418151855 
Previous Loss: 4.058918476104736
Error increased!

Epoch: 43 Step: 5825 
Current Va

Epoch: 56 Step: 7500 
Current Validation Loss: 4.336793899536133 
Previous Loss: 4.4207329750061035
Error increased!

Epoch: 56 Step: 7525 
Current Validation Loss: 4.304743766784668 
Previous Loss: 4.336793899536133
Error increased!

Epoch: 56 Step: 7550 
Current Validation Loss: 4.3617963790893555 
Previous Loss: 4.304743766784668

Epoch: 56 Step: 7575 
Current Validation Loss: 4.286497116088867 
Previous Loss: 4.3617963790893555
Error increased!

Epoch: 57 Step: 7600 
Current Validation Loss: 4.391557693481445 
Previous Loss: 4.286497116088867

Epoch: 57 Step: 7625 
Current Validation Loss: 4.508752346038818 
Previous Loss: 4.391557693481445

Epoch: 57 Step: 7650 
Current Validation Loss: 4.480581760406494 
Previous Loss: 4.508752346038818
Error increased!

Epoch: 57 Step: 7675 
Current Validation Loss: 4.439915657043457 
Previous Loss: 4.480581760406494
Error increased!

Epoch: 57 Step: 7700 
Current Validation Loss: 4.532635688781738 
Previous Loss: 4.439915657043457

Epoch: 58 St

Epoch: 70 Step: 9400 
Current Validation Loss: 4.908439636230469 
Previous Loss: 5.028847694396973
Error increased!

Epoch: 70 Step: 9425 
Current Validation Loss: 4.994662761688232 
Previous Loss: 4.908439636230469

Epoch: 71 Step: 9450 
Current Validation Loss: 4.951676368713379 
Previous Loss: 4.994662761688232
Error increased!

Epoch: 71 Step: 9475 
Current Validation Loss: 5.09348726272583 
Previous Loss: 4.951676368713379

Epoch: 71 Step: 9500 
Current Validation Loss: 4.971200942993164 
Previous Loss: 5.09348726272583
Error increased!

Epoch: 71 Step: 9525 
Current Validation Loss: 5.059657573699951 
Previous Loss: 4.971200942993164

Epoch: 71 Step: 9550 
Current Validation Loss: 4.913998126983643 
Previous Loss: 5.059657573699951
Error increased!

Epoch: 71 Step: 9575 
Current Validation Loss: 5.0150651931762695 
Previous Loss: 4.913998126983643

Epoch: 72 Step: 9600 
Current Validation Loss: 4.873663902282715 
Previous Loss: 5.0150651931762695
Error increased!

Epoch: 72 Step:

Epoch: 84 Step: 11300 
Current Validation Loss: 5.078343391418457 
Previous Loss: 4.910566329956055

Epoch: 85 Step: 11325 
Current Validation Loss: 5.233381271362305 
Previous Loss: 5.078343391418457

Epoch: 85 Step: 11350 
Current Validation Loss: 5.201307773590088 
Previous Loss: 5.233381271362305
Error increased!

Epoch: 85 Step: 11375 
Current Validation Loss: 5.0588698387146 
Previous Loss: 5.201307773590088
Error increased!

Epoch: 85 Step: 11400 
Current Validation Loss: 4.850531101226807 
Previous Loss: 5.0588698387146
Error increased!

Epoch: 85 Step: 11425 
Current Validation Loss: 5.100252151489258 
Previous Loss: 4.850531101226807

Epoch: 86 Step: 11450 
Current Validation Loss: 5.026523590087891 
Previous Loss: 5.100252151489258
Error increased!

Epoch: 86 Step: 11475 
Current Validation Loss: 4.99150276184082 
Previous Loss: 5.026523590087891
Error increased!

Epoch: 86 Step: 11500 
Current Validation Loss: 4.940922260284424 
Previous Loss: 4.99150276184082
Error increas

Epoch: 99 Step: 13175 
Current Validation Loss: 4.982461452484131 
Previous Loss: 5.0169358253479
Error increased!

Epoch: 99 Step: 13200 
Current Validation Loss: 4.960971832275391 
Previous Loss: 4.982461452484131
Error increased!

Epoch: 99 Step: 13225 
Current Validation Loss: 5.206162929534912 
Previous Loss: 4.960971832275391

Epoch: 99 Step: 13250 
Current Validation Loss: 5.244506359100342 
Previous Loss: 5.206162929534912

Epoch: 99 Step: 13275 
Current Validation Loss: 5.291563034057617 
Previous Loss: 5.244506359100342

Epoch: 99 Step: 13300 
Current Validation Loss: 5.412347793579102 
Previous Loss: 5.291563034057617



In [14]:
torch.save(model.state_dict(), 'hidden512_layers3_happy_birthday.net')

In [15]:
def predict_next_char(model, char, hidden=None, k=1):
    
    encoded_text = model.encoder[char]
    encoded_text = np.array([[encoded_text]])
    encoded_text = one_hot_encoder(encoded_text, len(model.all_chars))
    inputs = torch.from_numpy(encoded_text)
    
    if model.use_gpu:
        inputs = inputs.cuda()
        
    hidden = tuple([state.data for state in hidden])
    
    lstm_out, hidden = model(inputs, hidden)
    
    probs = F.softmax(lstm_out, dim=1).data
    
    if model.use_gpu:
        probs = probs.cpu()
        
    probs, index_positions = probs.topk(k)
    
    index_positions = index_positions.numpy().squeeze()
    
    probs = probs.numpy().flatten()
    
    probs = probs / probs.sum()
    
    char = np.random.choice(index_positions, p=probs)
    
    return model.decoder[char], hidden
    
    

In [16]:
def generate_text(model, size, seed='С днем рожденья ', k=1):
    
    if model.use_gpu:
        model.cuda()
    else:
        model.cpu()
        
    model.eval()
    
    output_chars = [c for c in seed]
    
    hidden = model.hidden_state(1)
    
    for char in seed:
        char, hidden = predict_next_char(model, char, hidden, k=k)
        
    output_chars.append(char)
    
    for i in range(size):
        
        char, hidden = predict_next_char(model, output_chars[-1], hidden, k=k)
        
        output_chars.append(char)
        
    return ''.join(output_chars)

In [17]:
print(generate_text(model, 3000, seed='С днем рожденья ', k=2))

С днем рожденья поздравляем
И от всей души желаем:
По-сибирски быть здоровым,
По-кавказски долго жить,
По-цыгански быть веселым
И по-русски водку пить!
С Днем рождения, мой милый!
С первым юбилеем!
Будь всегда удача!
И пусть костер большой любви
Горит, не угасая!
С любви и уважении,
В приятном окружении.

***

Невинность – это детская черта,
Она тебе с рождения дана.
И пусть костер большой любви
Горит, не угас,
И вот сегодня с этой друга
Идеди сероца и горя не запечали, Чтоб больше было светлых дней
И жить лет этак до 100.

***

Желаем здоровья – ведь часто его не хватает,
Желаем счастья и добра!

***

Любой юбилей – это чуточку грустно,
Не нежно воркуют —
Двух нежных сердец торжество
Хорошую найти.
И чтоб преграды редко
Встречались на пути.

***

Мысль поздравления дана.
И пусть бу те морошее из детства
И не оставят с ним с участьем
Прекрасной любви божество:
Два голубя нежно воркуют —
Двух нежных сердец торжество.

***

С законным беды прекрасен:
Каких в нем только не найдешь цветы,
